In [ ]:
import wandb
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
api = wandb.Api(timeout=120)
runs = api.runs("wilrop/IPRO_experiments")
env_id = "deep-sea-treasure-concave-v0"

In [ ]:
parent_runs = {
    'dqn': {
        'deep-sea-treasure-concave-v0': ['wilrop/IPRO_dqn_dst/1cdjwahh'],
        'minecart-v0': ['wilrop/IPRO/x2pv2xwv'],
        'mo-reacher-v4': ['wilrop/IPRO/12zmmwr8']
    },
    'a2c': {
        'deep-sea-treasure-concave-v0': ['wilrop/IPRO/nxiv5fqn'],
        'minecart-v0': ['wilrop/IPRO/84vg9vtt'],
        'mo-reacher-v4': ['wilrop/IPRO/1vfi17wo']
    },
    'ppo': {
        'deep-sea-treasure-concave-v0': ['wilrop/IPRO/1y9o463h'],
        'minecart-v0': ['wilrop/IPRO/y4lbmsh8'],
        'mo-reacher-v4': ['wilrop/IPRO/3706ujn8']
    }
}

run_hists = {'dqn': defaultdict(list),
             'a2c': defaultdict(list),
             'ppo': defaultdict(list), }

for run in runs:
    if run.config['env_id'] == env_id:
        name = run.name
        components = name.split('__')
        parent_run = components[1]
        alg = None
        for key, values in parent_runs.items():
            if parent_run in values[env_id]:
                alg = key
                break
        if alg is not None:
            run_hists[alg][parent_run].append(run.history(keys=['outer/hypervolume', 'outer/coverage']))
            print(f'Added run to {alg} - {parent_run}')

In [ ]:
def extract_iter_hist(hist):
    hypervolumes = list(hist['outer/hypervolume'].values)
    coverages = list(np.clip(list(hist['outer/coverage'].values), 0, 1))
    return hypervolumes, coverages

In [ ]:
run_data = {'dqn': defaultdict(list),
            'a2c': defaultdict(list),
            'ppo': defaultdict(list), }

for alg in run_hists:
    print(f"Extracting data for {alg}")
    for arg in run_hists[alg]:
        print(f"Extracting data for {alg} - {arg}")
        for seed, hist in enumerate(run_hists[alg][arg]):
            print(f'Run {seed}')
            hypervolumes, coverages = extract_iter_hist(hist)
            run_data[alg][arg].append((hypervolumes, coverages))

In [ ]:
best_data = {'ppo': [],
             'dqn': [],
             'a2c': []}

# Extract the best argument i.e. the argument with the largest mean final hypervolume
max_iter = 0

for alg in run_data:
    best_arg = None
    best_mean = -1
    best_std = -1
    for arg in run_data[alg]:
        hypervolumes = [tpl[0] for tpl in run_data[alg][arg]]
        final_hvs = [hv[-1] for hv in hypervolumes]
        arg_mean = np.mean(final_hvs)
        arg_std = np.std(final_hvs)
        print(f"Mean final hypervolume for {alg} - {arg}: {arg_mean}")

        if arg_mean > best_mean and len(run_data[alg][arg]) == 5:
            best_mean = arg_mean
            best_arg = arg
            best_std = arg_std
            max_iter = max(max_iter, max([len(hv) for hv in hypervolumes]))
    print(f"Best argument for {alg} is {best_arg} with final hypervolume {best_mean} +/- {best_std}")
    print('---')
    best_data[alg] = run_data[alg][best_arg]

In [ ]:
def fill_iterations(hypervolumes, coverages, max_iter):
    while len(hypervolumes) < max_iter:
        hypervolumes.append(hypervolumes[-1])
        coverages.append(coverages[-1])

In [ ]:
print(f"Max iterations: {max_iter}")
for alg in best_data:
    for seed, (hypervolumes, coverages) in enumerate(best_data[alg]):
        fill_iterations(hypervolumes, coverages, max_iter)

In [ ]:
# Make dictionaries with the data for all seeds.
for alg in best_data:
    hv_dict = {alg: [], 'Iteration': [], 'Seed': []}
    cov_dict = {alg: [], 'Iteration': [], 'Seed': []}

    for seed, (hypervolumes, coverages) in enumerate(best_data[alg]):
        hv_dict[alg].extend(hypervolumes)
        cov_dict[alg].extend(coverages)
        hv_dict['Iteration'].extend(range(max_iter))
        cov_dict['Iteration'].extend(range(max_iter))
        hv_dict['Seed'].extend([seed] * max_iter)
        cov_dict['Seed'].extend([seed] * max_iter)

    hv_df = pd.DataFrame.from_dict(hv_dict)
    cov_df = pd.DataFrame.from_dict(cov_dict)
    hv_df.to_csv(f'../utils/results/{alg}_{env_id}_hv.csv', index=False)
    cov_df.to_csv(f'../utils/results/{alg}_{env_id}_cov.csv', index=False)